<div style="font-size:15px; font-family:Roboto;background-color:#333333; color: white;">
    <img width="121" height="40" src="https://courier-images-web.imgix.net/static/img/white-logo.png?auto=compress,format&amp;fit=max&amp;w=undefined&amp;h=undefined&amp;dpr=2&amp;fm=png" alt="Zé delivery logo" class="css-0">  
    <h1 style="font-size:15px; font-family:Roboto;background-color:#333333; color: white;"><center><b>Walmart Store Sales Forecasting</b> | <a href="https://www.linkedin.com/in/guimarotto/" target="_blank" style="font-family:Roboto; color: #ffcd02;">Guilherme Lima</a></center></h1>


<div style="font-size:20px; font-family:Roboto; background-color:#F5F5F5; color: black;">
    <h1>Summary</h1>

 * <a href="#ch1" target="_blank" style="font-family:Roboto; color: #ffcd02;"><b>Abstract</b></a>
 * <a href="#ch2" target="_blank" style="font-family:verdana; color: #ffcd02;"><b>Exploratory Data Analysis</b></a>
     * <a href="#ch2_1" target="_blank" style="font-family:verdana; color: #ffcd02;"><b>Data Ingestion and Preparation</b></a>
     * <a href="#ch2_1" target="_blank" style="font-family:verdana; color: #ffcd02;"><b>Exploratory Analysis using Profile</b></a>
 * <a href="#ch3" target="_blank" style="font-family:verdana; color: #ffcd02;"><b>Feature Engineering</b></a>
     * <a href="#ch3_1" target="_blank" style="font-family:verdana; color: #ffcd02;"><b>Feature Preparation</b></a>
     * <a href="#ch3_3" target="_blank" style="font-family:verdana; color: #ffcd02;"><b>Feature Selection</b></a>  
 * <a href="#ch4" target="_blank" style="font-family:verdana; color: #ffcd02;"><b>Model Development</b></a>
     * <a href="#ch4_1" target="_blank" style="font-family:verdana; color: #ffcd02;"><b>Parameter Optimization</b></a>
     * <a href="#ch4_2" target="_blank" style="font-family:verdana; color: #ffcd02;"><b>Model Training and Tuning</b></a>
     * <a href="#ch4_3" target="_blank" style="font-family:verdana; color: #ffcd02;"><b>Model Validation</b></a>
 * <a href="#ch5" target="_blank" style="font-family:verdana; color: #ffcd02;"><b>Results</b></a>
 


<a id="ch1"></a>
<h1 id="ch1" style="font-family:Roboto;background-color:#333333; color: white;"> 
    <center><b><br>Abstract</b></center>
</h1>

<p style="font-size:15px; font-family:verdana; line-height: 1.7em">This notebook presents a experiment that forecast store sales based on Walmart historical data. The final Features selected were:</p>
    <ol style="font-size:15px; font-family:verdana; line-height: 1.7em"><li>
    <ol>Input Features
        <li>Store</li>
        <li>Dept</li>
        <li>IsHoliday</li>
        <li>Type</li>
        <li>Size</li>
        <li>Week</li>
        <li>Year</li>
    </ol>
</li>
<li>Target
    <ol>
        <li>Weekly_Sales</li>
    </ol>
</li>
</ol>
<div style="color:white;
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
        The final model = RandomForestRegressor(n_estimators=50, max_depth=27, max_features=6, min_samples_split=2, min_samples_leaf=1)   <br> The final RMSE error was 1171.018166.
    </p>
</div>

***

<a id="ch1_1"></a>
<div style="font-family:verdana; word-spacing:1.5px;">
    <h1 id="italic">
        Configuration and Import
    </h1>
    <br>
<br>
</div>

In [ ]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import warnings
warnings.filterwarnings("ignore") # ignoring annoying warnings

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor
import xgboost as xgb

enable_pandas_profilling = False
enable_tpot = False

<a id="ch2"></a>
<h1 id="basics" style="font-family:Roboto;background-color:#333333; color: white;"> 
    <center><b><br>Exploratory Data Analysis</b></center>
</h1>

<a id="ch2_1"></a>
<div style="font-family:verdana; word-spacing:1.5px;">
    <h1 id="italic">
        Data Ingestion and Preparation
    </h1>
    <ol>
        <li>Data Ingestion
            <ol>
                <li>Features Dataset</li>
                <li>Stores Dataset</li>
                <li>Training Dataset</li>
                <li>Testing Dataset</li>
                <li>Submission Dataset</li>
            </ol>
        </li>
        <li>Data Preparation
            <ol>
                <li>Join Features and Stores</li>
                <li>Verify datetypes</li>
                <li>Date represents Days or Weeks?</li>
                <li>Enrich Train and Test Datasets with Features/Stores Information</li>
            </ol>
        </li>
</ol>
</div>

In [ ]:
df_features = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
df_train = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
df_stores = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')
df_test = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
df_sample_submission = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')

In [ ]:
df_feat_stores = df_features.merge(df_stores, how='inner', on='Store')

In [ ]:
pd.DataFrame({'Type_Feat_Store': df_feat_stores.dtypes,'Type_Train': df_train.dtypes, 'Type_Test': df_test.dtypes})

<div style="color:white;
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
        After the ingestion, the field 'Date' on each dataset is a 'object' (string) type. We need to convert them to datetime for them to be better used in the future.
    </p>
</div>

In [ ]:
df_feat_stores.Date = pd.to_datetime(df_feat_stores.Date)
df_train.Date = pd.to_datetime(df_train.Date)
df_test.Date = pd.to_datetime(df_test.Date)

In [ ]:
len(df_feat_stores.Date.unique())/52

<div style="color:white;
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
        Since we verified that the 'Date' field doesn't represent the day itself, but each week, ending every friday, from 2010-02-05 to 2012-11-01, we will create to new features: (1) 'Week' and (2) 'Year'. 
    </p>
</div>


In [ ]:
df_feat_stores['Week'] = df_feat_stores.Date.dt.week 
df_feat_stores['Year'] = df_feat_stores.Date.dt.year

<div style="color:white;
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
      By now, the training and test datasets will be the inner join with repectively, train and test data enriched with features and store dataframe.
    </p>
</div>

In [ ]:
df_train_feats = df_train.merge(df_feat_stores, how='inner', on=['Store','Date','IsHoliday']).sort_values(by=['Store','Dept','Date']).reset_index(drop=True)
df_test_feats = df_test.merge(df_feat_stores, how='inner', on=['Store','Date','IsHoliday']).sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

In [ ]:
del df_feat_stores, df_train, df_test

<a id="ch2_2"></a>
<div style="font-family:verdana; word-spacing:1.5px;">
    <h1 id="italic">
        Data Preparation
    </h1>
    <ol>
        <li>Missing Values
            <ol>
                <li>Drop cols with more than 60% of missing values</li>
            </ol>
        </li>
        <li>Person Correlation - Positive Correlation indicates that when one variable increase, the other also does. Negative is the opposite.
            <ol>
                <li>0: no correlation</li>
                <li>from 0 to +/-0.3: weak correlation</li>
                <li>from +/-0.3 to +/-0.7: moderate correlaton</li>
                <li>from +/-0.7 to +/-1: strong correlation</li>
            </ol>
        </li>
</ol>
</div>
<div style="color:white;
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
      Ideally we want to use features with moderate and strong positive correlations with 'Weekly_sales' and with less than 60% missing values.
    </p>
</div>

In [ ]:
#Generate the report. We would use the mpg dataset as sample, title parameter for naming our report, and explorative parameter set to True for Deeper exploration.
if enable_pandas_profilling:
    profile_train = ProfileReport(df_train_feats, title='Training Dataset Report', explorative = True)
    profile_train.to_file(output_file='profile_train.html')
    profile_train

* MarkDown1 has 270889 (64.3%) missing values
* MarkDown2 has 310322 (73.6%) missing values
* MarkDown3 has 284479 (67.5%) missing values
* MarkDown4 has 286603 (68.0%) missing values
* MarkDown5 has 270138 (64.1%) missing values

<div style="color:white;
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
        Since MarkDown' 1 to 5 have more than 60% of missing values, aren't strong correlated to 'Weekly_Sales' and they are anonymized, i.e. it's difficult to know what they mean, we can drop them.
    </p>
</div>

In [ ]:
df_train_feats = df_train_feats.drop(columns=['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'])
df_test_feats = df_test_feats.drop(columns=['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'])

In [ ]:
sns.set(style="white")
corr = df_train_feats.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
f, ax = plt.subplots(figsize=(20, 15))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
plt.title('Correlation Matrix', fontsize=18)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

plt.show()

In [ ]:
corr['Weekly_Sales'].sort_values()

<div style="color:white;
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
'Fuel_Price', 'Temperature', 'Date', 'CPI', 'Unemployment' have week negative correlation with 'Weekly_sales', so they will be dropped. 
<br>Also, 'Fuel_Price' is strong correlated to 'Year'. We will keep 'Year', and drop 'Fuel_price', because it differentiate same Weeks Numbers for Sales.</p>
</div>

In [ ]:
df_train_feats = df_train_feats.drop(columns=['Fuel_Price', 'Temperature','Date', 'CPI', 'Unemployment'])
df_test_feats = df_test_feats.drop(columns=['Fuel_Price', 'Temperature','Date', 'CPI', 'Unemployment'])

In [ ]:
sns.set(style="white")
corr = df_train_feats.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
f, ax = plt.subplots(figsize=(20, 15))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
plt.title('Correlation Matrix', fontsize=18)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

plt.show()

In [ ]:
corr['Weekly_Sales'].sort_values()

***

<a id="ch3"></a>
<h1 id="basics" style="font-family:Roboto;background-color:#333333; color: white;"> 
    <center><b><br>Feature Engineering</b></center>
</h1>

<a id="ch3_1"></a>
<div style="font-family:verdana; word-spacing:1.5px;">
    <h1 id="italic">
        Feature Preprocessing
    </h1>
    <ol>
        <li>Holidays Analysis
            <ol>
                <li>When did the holydays happened?</li>
                <li>IsHolyday covers all US Holydays?</li>
            </ol>
        </li>
        <li>Features types
            <ol>
                <li>Can we have only numerical features?</li>
                <li>What Type stands for?</li>
            </ol>
        </li>
</ol>
</div>
<div style="color:white;
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
      Ideally we want to use numerical features.
    </p>
</div>

In [ ]:
df_sales_2010_weekly = df_train_feats[df_train_feats.Year==2010]['Weekly_Sales'].groupby(df_train_feats['Week']).sum()
df_sales_2011_weekly = df_train_feats[df_train_feats.Year==2011]['Weekly_Sales'].groupby(df_train_feats['Week']).sum()
df_sales_2012_weekly = df_train_feats[df_train_feats.Year==2012]['Weekly_Sales'].groupby(df_train_feats['Week']).sum()

In [ ]:
plt.figure(figsize=(20,8))
sns.lineplot(df_sales_2010_weekly.index, df_sales_2010_weekly.values)
sns.lineplot(df_sales_2011_weekly.index, df_sales_2011_weekly.values)
sns.lineplot(df_sales_2012_weekly.index, df_sales_2012_weekly.values)
plt.grid()
plt.xticks(np.arange(1, 53, step=1))
plt.legend(['2010', '2011', '2012'], loc='best', fontsize=16)
plt.title('Overall Weekly Sales - Per Year', fontsize=18)
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Week', fontsize=16)
plt.show()

In [ ]:
df_train_feats[df_train_feats.IsHoliday].Week.unique()

As we can see, there is one important Holiday not included in 'IsHoliday'. It's the Easter Day. It is always in a Sunday, but can fall on different weeks.

In 2010 is in Week 13
In 2011, Week 16
Week 14 in 2012
and, finally, Week 13 in 2013 for Test set
So, we can change to 'True' these Weeks in each Year.

In [ ]:
df_train_feats.loc[(df_train_feats.Year==2010) & (df_train_feats.Week==13), 'IsHoliday'] = True
df_train_feats.loc[(df_train_feats.Year==2011) & (df_train_feats.Week==16), 'IsHoliday'] = True
df_train_feats.loc[(df_train_feats.Year==2012) & (df_train_feats.Week==14), 'IsHoliday'] = True
df_test_feats.loc[(df_test_feats.Year==2013) & (df_test_feats.Week==13), 'IsHoliday'] = True

In [ ]:
weekly_sales_mean = df_train_feats['Weekly_Sales'].groupby(df_train_feats['Week']).mean()
plt.figure(figsize=(20,8))
sns.lineplot(weekly_sales_mean.index, weekly_sales_mean.values)
plt.grid()
plt.xticks(np.arange(1, 53, step=1))
plt.legend(['Mean'], loc='best', fontsize=16)
plt.title('Mean Weekly Sales', fontsize=18)
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Week', fontsize=16)
plt.axvline(6, color='red')
plt.axvline(13, color='red')
plt.axvline(14, color='red')
plt.axvline(36, color='red')
plt.axvline(47, color='red')
plt.axvline(52, color='red')
plt.show()

In [ ]:
df_train_feats.Type.unique()

In [ ]:
fig = plt.figure(figsize=(20,8))
gs = GridSpec(1,2)
sns.boxplot(y=df_train_feats.Weekly_Sales, x=df_train_feats.Type, ax=fig.add_subplot(gs[0,0]))
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Type', fontsize=16)
sns.stripplot(y=df_train_feats.Weekly_Sales, x=df_train_feats.Type, ax=fig.add_subplot(gs[0,1]))
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Type', fontsize=16)
fig.show()

In [ ]:
def nominal_to_ordinal(x):
    if x == 'A':
        return 3
    elif x == 'B':
        return 2
    else:
        return 1
    
df_train_feats.Type = df_train_feats.Type.apply(nominal_to_ordinal)
df_test_feats.Type = df_test_feats.Type.apply(nominal_to_ordinal)

In [ ]:
df_train_feats.Type.unique()

In [ ]:
df_test_feats.Type.unique()

In [ ]:
fig = plt.figure(figsize=(20,8))
gs = GridSpec(1,2)
sns.boxplot(y=df_train_feats.Weekly_Sales, x=df_train_feats.Type, ax=fig.add_subplot(gs[0,0]))
plt.ylabel('Sales', fontsize=16)
plt.xlabel('TypeOrd', fontsize=16)
sns.stripplot(y=df_train_feats.Weekly_Sales, x=df_train_feats.Type, ax=fig.add_subplot(gs[0,1]))
plt.ylabel('Sales', fontsize=16)
plt.xlabel('TypeOrd', fontsize=16)
fig.show()

In [ ]:
def bool_to_num(x):
    if x:
        return 1
    else:
        return 0
    
df_train_feats.IsHoliday = df_train_feats.IsHoliday.apply(bool_to_num)
df_test_feats.IsHoliday = df_test_feats.IsHoliday.apply(bool_to_num)

<a id="ch3_3"></a>
<div style="font-family:verdana; word-spacing:1.5px;">
    <h1 id="italic">
        Feature Selection
    </h1>
</div>
<div style="
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
            <ol><li>Features
            <ol>
                <li>Store</li>
                <li>Dept</li>
                <li>IsHoliday</li>
                <li>Type</li>
                <li>Size</li>
                <li>Week</li>
                <li>Year</li>
            </ol>
        </li>
        <li>Target
            <ol>
                <li>Weekly_Sales</li>
            </ol>
        </li>
</ol><br></p>
</div>

In [ ]:
df_train_feats.head()

In [ ]:
df_test_feats.head()

In [ ]:
df_train_feats.IsHoliday.unique()

***

<a id="ch4"></a>
<h1 id="basics" style="font-family:Roboto;background-color:#333333; color: white;"> 
    <center><b><br>Model Development</b></center>
</h1>

<a id="ch4_1"></a>
<div style="font-family:verdana; word-spacing:1.5px;">
    <h1 id="italic">Parameter Optimization</h1>
</div>
<div style="
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
            In order to get a better Parameter Optimization and also select the model algorithm that better suits our features I ran TPOT Regressor.</p>
</div>

<img  src="https://github.com/EpistasisLab/tpot/raw/master/images/tpot-pipeline-example.png" alt="Tpot mechanism" class="css-0">
from  <a href="https://github.com/EpistasisLab/tpot" target="_blank" style="font-family:Roboto; color: #ffcd02;">https://github.com/EpistasisLab/tpot</a>

In [ ]:
X_train = df_train_feats.drop('Weekly_Sales', axis= 1).values
y_train = df_train_feats.Weekly_Sales.values

In [ ]:
len(X_train), len(y_train) 

In [ ]:
# TPOT takes 10hours to get a good parameters estimation so it is set False here.
if enable_tpot:
    X_tpot_train, X_tpot_test, y_tpot_train, y_tpot_test = train_test_split(X_train,  y_train, train_size=0.75, test_size=0.25, random_state=42)
    tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42)
    tpot.fit(X_tpot_train, y_tpot_train)
    print(tpot.score(X_tpot_test, y_tpot_test))
    tpot.export('tpot_pipeline.py')

<div style="
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
            TPOT Regressor estimation delivered the following params: n_estimators=100, max_depth=27, max_features=6, min_samples_split=8, min_samples_leaf=1.</p>
</div>

<a id="ch4_2"></a>
<div style="font-family:verdana; word-spacing:1.5px;">
    <h1 id="italic">
Model Training and Tuning
    </h1>
    <br>
    <p>
        From Tpot estimation I will try to tune the model in order to reduce the RMSE.<br>
</div>

In [ ]:
model = RandomForestRegressor(n_estimators=100, max_depth=27, max_features=6, min_samples_split=8, min_samples_leaf=1)
model.fit(X_train, y_train)
preds = model.predict(X_train)
rmse_all = np.sqrt(mean_squared_error(y_train, preds))
print("RMSE: %f" % (rmse_all))
del model

In [ ]:
model = RandomForestRegressor(n_estimators=50, max_depth=27, max_features=6, min_samples_split=8, min_samples_leaf=1)
model.fit(X_train, y_train)
preds = model.predict(X_train)
rmse_all = np.sqrt(mean_squared_error(y_train, preds))
print("RMSE: %f" % (rmse_all))
del model

In [ ]:
model = RandomForestRegressor(n_estimators=50, max_depth=27, max_features=6, min_samples_split=4, min_samples_leaf=1)
model.fit(X_train, y_train)
preds = model.predict(X_train)
rmse_all = np.sqrt(mean_squared_error(y_train, preds))
print("RMSE: %f" % (rmse_all))
del model

In [ ]:
model = RandomForestRegressor(n_estimators=50, max_depth=27, max_features=6, min_samples_split=3, min_samples_leaf=1)
model.fit(X_train, y_train)
preds = model.predict(X_train)
rmse_all = np.sqrt(mean_squared_error(y_train, preds))
print("RMSE: %f" % (rmse_all))
del model

In [ ]:
model = RandomForestRegressor(n_estimators=50, max_depth=27, max_features=6, min_samples_split=2, min_samples_leaf=1)
model.fit(X_train, y_train)
preds = model.predict(X_train)
rmse_all = np.sqrt(mean_squared_error(y_train, preds))
print("RMSE: %f" % (rmse_all))

<div style="
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
            The final model = RandomForestRegressor(n_estimators=50, max_depth=27, max_features=6, min_samples_split=2, min_samples_leaf=1)</p>
</div>

<a id="ch4_3"></a>
<div style="font-family:verdana; word-spacing:1.5px;">
    <h1 id="italic">
Model Validation
    </h1>
    <br>
    <p>
        Let's use the Test Dataset and build our submission sample.<br>
</div>

In [ ]:
len(df_test_feats), len(df_sample_submission)

In [ ]:
df_test_feats.info()

In [ ]:
predicted = model.predict(df_test_feats)

In [ ]:
df_sample_submission['Weekly_Sales'] = predicted
df_sample_submission.to_csv('submission.csv',index=False)

***

<a id="ch5"></a>
<h1 id="ch5" style="font-family:Roboto;background-color:#333333; color: white;"> 
<center><b><br>Results</b></center>
</h1>

<div style="
       display:fill;
       border-radius:5px;
       background-color:#ffcd02;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:#333337;">
            The final model = RandomForestRegressor(n_estimators=50, max_depth=27, max_features=6, min_samples_split=2, min_samples_leaf=1)   <br> The final RMSE error was 1171.018166 </p>
</div>
